# Assignment 4 
by Himani Anil Deshpande

In [1]:
pip install py4j

     |████████████████████████████████| 198 kB 5.3 MB/s 


In [2]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 37 kB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=b609ff2596f776804ca6439644627e45e7bd365cfb07084982ae1b31568e24de
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


Connecting to Spark Cluster

In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, IntegerType, DateType
from pyspark.sql.functions import col, asc,desc
import json
import csv
from io import StringIO
from pyspark.sql import HiveContext
sc = SparkContext()

Load fake_data.csv

In [6]:

# datafile = "file:///Users/himanideshpande/miniforge3/envs/ENV_SPARK/SparkInstallation/spark-3.2.1-bin-hadoop2.7/IPYNB/Fake_data.csv"
datafile = "/content/Fake_data.csv"
# input_json = sc.textFile(datafile)
def loadRecord(line):
    """Parse a CSV line"""
    input = StringIO(line)
    reader = csv.DictReader(input, fieldnames=["index","Birth_Country","Email","First_Name","Income","Job","Last_name","Loan_Approved","SSN"])
    return next(reader)

input_data = sc.textFile(datafile).map(loadRecord)


In [7]:
rdd = sc.parallelize(input_data.collect())

In [8]:
# rdd.collect()

spark = SparkSession(sc)

In [9]:
df =  rdd.toDF()

In [10]:
df.printSchema()

root
 |-- Birth_Country: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- First_Name: string (nullable = true)
 |-- Income: string (nullable = true)
 |-- Job: string (nullable = true)
 |-- Last_name: string (nullable = true)
 |-- Loan_Approved: string (nullable = true)
 |-- SSN: string (nullable = true)
 |-- index: string (nullable = true)



In [11]:
df = df.withColumn('Income', col('Income').cast(IntegerType()))

In [12]:
df.printSchema()

root
 |-- Birth_Country: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- First_Name: string (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Job: string (nullable = true)
 |-- Last_name: string (nullable = true)
 |-- Loan_Approved: string (nullable = true)
 |-- SSN: string (nullable = true)
 |-- index: string (nullable = true)



In [13]:
type(rdd)

pyspark.rdd.RDD

Question 1. 
Find birth country which has highest amount of people 

In [16]:
df.createOrReplaceTempView("Fake_data")

sqlDF = spark.sql("SELECT COunt(Birth_Country), Birth_Country FROM Fake_data group By Birth_Country order by 1 DESC Limit 1")
sqlDF.show()


+--------------------+-------------+
|count(Birth_Country)|Birth_Country|
+--------------------+-------------+
|                  91|        Korea|
+--------------------+-------------+



Question 2
Find average income of people who are born in united states of america 

In [21]:

sqlDF = spark.sql("SELECT Avg(tb.Income), tb.Birth_Country FROM Fake_data tb  group by tb.Birth_Country having tb.Birth_Country = 'United States of America' ")
sqlDF.show()

+------------------+--------------------+
|       avg(Income)|       Birth_Country|
+------------------+--------------------+
|208759.82352941178|United States of ...|
+------------------+--------------------+



Question 3 How many people has income over 100,000 but their loan is not approved. 

In [23]:

sqlDF = spark.sql("SELECT COUNT(*) FROM Fake_data tb where tb.Income > 100000 and tb.Loan_Approved = False ")
sqlDF.show()

+--------+
|count(1)|
+--------+
|    4009|
+--------+



Question 4 Find top 10 people with highest income in United States of america. (Print their names, income and jobs)

In [27]:

sqlDF = spark.sql("SELECT tb.First_Name,tb.Last_name, tb.Job, tb.Income  FROM Fake_data tb where tb.Birth_Country = 'United States of America' order by tb.Income DESC Limit 10 ")
sqlDF.show()

+----------+---------+--------------------+------+
|First_Name|Last_name|                 Job|Income|
+----------+---------+--------------------+------+
|    Alyssa|   Miller|Amenity horticult...|482588|
|    Hunter|    Walls|Psychologist, pri...|468946|
|      Rose|Henderson|Adult guidance wo...|426115|
|  Danielle|  Leonard|Furniture conserv...|389810|
|     Terry|    Klein|       Meteorologist|380410|
|     Cindy|   Newton|Research scientis...|370322|
|     Scott| Mitchell|       Art therapist|368913|
|   Christy| Sandoval|      Engineer, land|355150|
|     Kelly| Reynolds|           Press sub|341448|
|  Kristina|    Smith|           Herbalist|338804|
+----------+---------+--------------------+------+



Question 5 How many number of distinct jobs are there?

In [28]:

sqlDF = spark.sql("SELECT count(DISTINCT(tb.Job)) FROM Fake_data tb  ")
sqlDF.show()

+-------------------+
|count(DISTINCT Job)|
+-------------------+
|                640|
+-------------------+



Question 6 How many writers earn less than 100,000? 

In [29]:

sqlDF = spark.sql("SELECT count(*) FROM Fake_data tb where tb.Income <100000 and tb.Job = 'Writer' ")
sqlDF.show()

+--------+
|count(1)|
+--------+
|       5|
+--------+



Reference

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html#:~:text=Spark%20SQL%20is%20a%20Spark,information%20to%20perform%20extra%20optimizations.